In [8]:
!pip3 install mnist
!pip3 install minisom

# imports
import matplotlib.pyplot as plt
import numpy as np
import mnist
import scipy.misc
import math
import seaborn as sns

from matplotlib.colors import ListedColormap
from matplotlib import cm

from sklearn import datasets,metrics
from sklearn.datasets import fetch_20newsgroups, fetch_rcv1
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from sklearn.decomposition import PCA, TruncatedSVD, KernelPCA
from sklearn.manifold import TSNE
import umap
from minisom import MiniSom


from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans, SpectralClustering, MiniBatchKMeans

from sklearn.metrics.pairwise import euclidean_distances

from joblib import Parallel, delayed
from tqdm import tqdm

!pip install ipython-autotime
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 11.2 s (started: 2021-02-06 19:08:39 -05:00)


In [9]:
# Load Digits dataset
digits = datasets.load_digits()
digits_n_samples = len(digits.images)
data = digits.images.reshape((digits_n_samples, -1))
digits_X_train, digits_X_test, digits_y_train, digits_y_test = train_test_split(data, digits.target, test_size=0.3, shuffle=True)


# Load full MNIST dataset
MNIST_X_train = mnist.train_images()
MNIST_X_train = MNIST_X_train.reshape((len(MNIST_X_train), -1)) 
MNIST_y_train = mnist.train_labels()

MNIST_X_test = mnist.test_images()
MNIST_X_test = MNIST_X_test.reshape((len(MNIST_X_test), -1)) 
MNIST_y_test = mnist.test_labels()

total_samples = len(MNIST_X_train) + len(MNIST_X_test)
blob_test_size = len(MNIST_X_test)/total_samples



#Load the 20NewsGroups
#Load the 20NewsGroups
vectorizer = TfidfVectorizer(min_df=5, stop_words='english') 

newsgroups_train = fetch_20newsgroups(subset='all')
newsgroups_X_train = vectorizer.fit_transform(newsgroups_train.data).toarray()
newsgroups_y_train = newsgroups_train.target
print(newsgroups_X_train.shape)




#Load the RCV dataset
#rcv1_train = fetch_rcv1(subset='train')
#rcv1_X_train = newsgroups_train.data
#rcv1_y_train = newsgroups_train.target

#rcv1_test = fetch_rcv1(subset='test')
#rcv1_X_test = newsgroups_test.data
#rcv1_y_test = newsgroups_test.target




(18846, 34880)
time: 8.51 s (started: 2021-02-06 19:08:50 -05:00)


In [10]:
# k-Means clustering
def kmeans(X_train, n_clusters):
  kmeans = KMeans(n_clusters=n_clusters, random_state=42)
  y_pred_train = kmeans.fit_predict(X_train)

  return y_pred_train
  

time: 567 µs (started: 2021-02-06 19:08:58 -05:00)


In [11]:
# Spectral clustering 
def spectral(X_train, n_clusters):
  #med = np.median(euclidean_distances(X_train))
  spectral = SpectralClustering(n_clusters=n_clusters, random_state=0)
  y_pred_train = spectral.fit_predict(X_train)
  #y_pred_test = spectral.fit_predict(X_test)

  return y_pred_train#y_pred_test


time: 849 µs (started: 2021-02-06 19:08:58 -05:00)


In [12]:
def purity_score(y_true, y_pred):
    # compute contingency matrix (also called confusion matrix)
    contingency_matrix = metrics.cluster.contingency_matrix(y_true, y_pred)
    # return purity
    return np.sum(np.amax(contingency_matrix, axis=0)) / np.sum(contingency_matrix) 

# Evaluation function
def evaluate(X, y, y_pred):
    print("Accurary Score: %0.3f" % purity_score(y, y_pred))
    print("Adjusted Mutual Information Score: %0.3f" % metrics.adjusted_mutual_info_score(y, y_pred))
    print("Adjusted Rand Index Score: %0.3f" % metrics.adjusted_rand_score(y, y_pred))
    print("Normalized Mutual Information Score: %0.3f" % metrics.normalized_mutual_info_score(y, y_pred))

    print("Homogeneity: %0.3f" % metrics.homogeneity_score(y, y_pred))
    print("Completeness: %0.3f" % metrics.completeness_score(y, y_pred))
    print("V-measure: %0.3f" % metrics.v_measure_score(y, y_pred))
    #print(np.unique(y_pred))
    print("Silhouette Coefficient: %0.3f" % metrics.silhouette_score(X, y_pred, sample_size=1000))
    print()



time: 1.32 ms (started: 2021-02-06 19:08:58 -05:00)


In [13]:
# dimension reduction


def pca_reduce(X_train, dim):
  pca = None
  if scipy.sparse.issparse(X_train):
    pca = TruncatedSVD(n_components=dim)
  else:
    pca = PCA(n_components=dim)
  X_train_pca = pca.fit_transform(X_train)

  return X_train_pca


def tsne_reduce(X_train, dim):
  tsne = TSNE(n_components= dim)
  X_train_tsne = tsne.fit_transform(X_train)

  return X_train_tsne 

def umap_reduce(X_train, dim):
  umapr = umap.UMAP(n_components=700, n_neighbors = 25, min_dist = 1,  n_jobs=1, verbose = True)
  X_train_umap = umapr.fit_transform(X_train)
  umapr = umap.UMAP(n_components=dim, n_neighbors = 25, min_dist = 1,  n_jobs=1, verbose = True)
  X_train_umap = umapr.fit_transform(X_train_umap)
  return X_train_umap


def kernel_pcarbf_reduce(X_train, dim):
  kernal_pca = KernelPCA(n_components=dim, kernel='rbf', gamma=  2*(1/19)**2)
  X_train_kpca = kernal_pca.fit_transform(X_train)
  
  return X_train_kpca








def dimension_reduce(dimensions, X_train, y_train, reduce, n_clusters):
  #y_pred_train = kmeans(X_train, n_clusters)
  #print("Original KMeans")
  #evaluate(X_train,y_train,y_pred_train) 

  
  
  for dim in dimensions:
    #reduction
    print("Reduce")
    X_train_pca = reduce(X_train,  dim)
    
    y_pred_train  = kmeans(X_train_pca, n_clusters)
    print("KMeans dimension reduce: " + str(dim))
    evaluate(X_train_pca,y_train,y_pred_train)





dimension_reduce([500], newsgroups_X_train , newsgroups_y_train,  umap_reduce, 20)

Reduce
UMAP(dens_frac=0.0, dens_lambda=0.0, min_dist=1, n_components=700, n_jobs=1,
     n_neighbors=25, verbose=True)
Construct fuzzy simplicial set
Sat Feb  6 19:09:03 2021 Finding Nearest Neighbors
Sat Feb  6 19:09:03 2021 Building RP forest with 12 trees
Sat Feb  6 19:09:52 2021 NN descent for 14 iterations
	 1  /  14
	 2  /  14
	 3  /  14
	 4  /  14
	 5  /  14
	 6  /  14
	Stopping threshold met -- exiting after 6 iterations
Sat Feb  6 19:16:25 2021 Finished Nearest Neighbor Search
Sat Feb  6 19:16:26 2021 Construct embedding
	completed  0  /  200 epochs
	completed  20  /  200 epochs
	completed  40  /  200 epochs
	completed  60  /  200 epochs
	completed  80  /  200 epochs
	completed  100  /  200 epochs
	completed  120  /  200 epochs
	completed  140  /  200 epochs
	completed  160  /  200 epochs
	completed  180  /  200 epochs
Sat Feb  6 19:26:39 2021 Finished embedding
UMAP(dens_frac=0.0, dens_lambda=0.0, min_dist=1, n_components=500, n_jobs=1,
     n_neighbors=25, verbose=True)
Cons

In [14]:
# Visualization general

def visualize(X_train, X_test, y_train, y_test, reduce, title, n_clusters):
 
  colors = []
  colors += cm.get_cmap("Set3").colors
  colors += cm.get_cmap("Set2").colors
  my_cmap = ListedColormap(colors)
  
  X_train_r, X_test_r = reduce(X_train, X_test, 2)
 

  kmy_pred_train, kmy_pred_test, train_centers, test_centers = kmeans(X_train_r, X_test_r, y_train, y_test, n_clusters)
  spy_pred_train, spy_pred_test = spectral(X_train_r, X_test_r, y_train, y_test, n_clusters)

  
  fig=plt.figure(figsize=(14,10))
  # Train orginial labels
  plt.subplot(2, 3, 1)
  plt.gca().set_title('Train Data With Orginial Labels')
  plt.scatter(X_train_r[:, 0], X_train_r[:, 1], c= y_train, s=20, cmap=my_cmap)

  # Train kmeans labels
  plt.subplot(2, 3, 2)
  plt.gca().set_title('Train Data With KMeans Clusters')
  plt.scatter(X_train_r[:, 0], X_train_r[:, 1], c= kmy_pred_train, s=20, cmap=my_cmap)
  plt.scatter(train_centers [:, 0], train_centers [:, 1], c='black', s=100, alpha=0.8)

  # Train spectral labels
  plt.subplot(2, 3, 3)
  plt.gca().set_title('Train Data With Spectral Clusters')
  plt.scatter(X_train_r[:, 0], X_train_r[:, 1], c= spy_pred_train, s=20, cmap=my_cmap)


  # Test orginial labels
  plt.subplot(2, 3, 4)
  plt.gca().set_title('Test Data With Orginial Labels')
  plt.scatter(X_test_r[:, 0], X_test_r[:, 1], c= y_test, s=20,cmap=my_cmap)

  # Test kmeans labels
  plt.subplot(2, 3, 5)
  plt.gca().set_title('Test Data With KMeans Clusters')
  plt.scatter(X_test_r[:, 0], X_test_r[:, 1], c= kmy_pred_test, s=20,cmap=my_cmap)
  plt.scatter(test_centers[:, 0], test_centers[:, 1], c='black', s=100, alpha=0.8)

  # Test spectral labels
  plt.subplot(2, 3, 6)
  plt.gca().set_title('Test Data With Spectral Clusters')
  plt.scatter(X_test_r[:, 0], X_test_r[:, 1], c= spy_pred_test, s=20, cmap=my_cmap)

  fig.suptitle(title + " Reduction", fontsize=16)

  fig.tight_layout()
  fig.subplots_adjust(top=0.93)
  plt.show()



#visualize(newsgroups_X_train , newsgroups_X_test , newsgroups_y_train, newsgroups_y_test, umap_reduce, "", 20)


time: 2.03 ms (started: 2021-02-06 19:32:50 -05:00)


In [15]:
# Digits reduction 
#dimension_reduce([48, 32, 16, 8, 4, 2], digits_X_train, digits_X_test, digits_y_train, digits_y_test, kernel_pcarbf_reduce, 10)
#visualize(digits_X_train, digits_X_test, digits_y_train, digits_y_test, som_reduce, "SOM", 10)

#visualize(digits_X_train, digits_X_test, digits_y_train, digits_y_test, tsne_reduce, "TSNE", 10)

time: 2.86 ms (started: 2021-02-06 19:32:50 -05:00)


In [ ]:
# fetch 20 reduction 
#dimension_reduce([ 1000, 500, 300, 200, 100, 50, 20], newsgroups_X_train , newsgroups_X_test , newsgroups_y_train , newsgroups_y_test, 20)

#PCA
#visualize(newsgroups_X_train , newsgroups_X_test , newsgroups_y_train, newsgroups_y_test, pca_reduce, "PCA", 20)

In [ ]:
dimension_reduce([7500, 5000, 2500, 1000, 500, 250, 50, 20], newsgroups_X_train , newsgroups_X_test , newsgroups_y_train, newsgroups_y_test, umap_reduce, 20)

In [ ]:
dimension_reduce([7500, 5000, 2500, 1000, 500, 250, 50, 20], newsgroups_X_train , newsgroups_X_test , newsgroups_y_train, newsgroups_y_test, pca_reduce, 20)

In [ ]:
# rcv1 reduction
#dimension_reduce([ 1000, 500, 300, 200, 100, 50, 20], rcv1_X_train , rcv1_X_test , rcv1_y_train , rcv1_y_test, 103)
#visualize_pca(rcv1_X_train , rcv1_X_test , rcv1_y_train , rcv1_y_test, 103)
#visualize_tsne(rcv1_X_train , rcv1_X_test , rcv1_y_train , rcv1_y_test, 103, True)